In [7]:
import numpy as np
import scipy as sp
from scipy import linalg as la
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter

In [29]:
def set_ticks(ax, xMaj, xMin, yMaj, yMin):
    ax.xaxis.set_major_locator(MultipleLocator(xMaj))
    ax.xaxis.set_minor_locator(AutoMinorLocator(xMin))
    ax.yaxis.set_major_locator(MultipleLocator(yMaj))
    ax.yaxis.set_minor_locator(AutoMinorLocator(yMin))
    ax.tick_params(which='major', width=1.0, length=10, direction="in", labelsize=12)
    ax.tick_params(which='minor', width=1.0, length=5, direction="in", labelsize=12)

1\. PCA on 3D dataset

* Generate a dataset simulating 3 features, each with N entries (N being ${\cal O}(1000)$). Each feature is made by random numbers generated according the normal distribution $N(\mu,\sigma)$ with mean $\mu_i$ and standard deviation $\sigma_i$, with $i=1, 2, 3$. Generate the 3 variables $x_{i}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues using the eigendecomposition of the covariance matrix
* Find the eigenvectors and eigenvalues using the SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained
* Redefine the data according to the new basis from the PCA
* Plot the data, in both the original and the new basis. The figure should have 2 rows (the original and the new basis) and 3 columns (the $[x_0, x_1]$, $[x_0, x_2]$ and $[x_1, x_2]$ projections) of scatter plots.

In [49]:
# generate dataset
N = 1000
x1 = np.random.normal(0, 1, N)
x2 = np.random.normal(0, 3, N) + x1
x3 = 2 * x1 + x2
data = np.array([x1, x2, x3])

# get covariance
cov = np.cov(data)
print("The covariance matrix is:\n", cov)

'''
    Eigendecomposition.
'''

# get eigenvalues and eigenvectors
egl, egv = la.eig(cov)

# sort them in order to match the latter SVD 
index = np.argsort(egl)[::-1]
egl = egl[index]
egv = egv[:, index]
egl_real = np.real_if_close(egl)
print("\nEIGENDECOMPOSITION:")
print("-Eigenvalues:\n", egl, "->", egl_real)
print("-Eigenvectors:\n", egv)

'''
    Perform the eigendecomposition using the SVD.
    The third eigenvalue is sensibly different. If one uses
    the np.allclose() method with a high precision of the 
    orderd o(1.e-20), the eingevalues are in fact different.
'''

# perform the Single Value Decomposition
U, S, V = la.svd(data)
egl_svd = S**2 / (N - 1)
egv_svd = U
print("\nEIGENDECOMPOSITION USING SVD:")
print("-Eigenvalues:\n", egl_svd)
print("-Eigenvectors:\n", egv_svd, "\n")

#check = np.allclose(egl, egl_svd, 1.e-20)
check = np.allclose(egl, egl_svd, 1.e-2)
if check:
    print("The two procedures approximately yield the same eigenvalues.")
check_egv = np.allclose(egv, egv_svd, 1.e-2)
if check_egv:
    print("The two procedures approximately yield the same eigenvectors.")

The covariance matrix is:
 [[ 1.03580401  1.13947062  3.21107863]
 [ 1.13947062  9.82037416 12.0993154 ]
 [ 3.21107863 12.0993154  18.52147267]]

EIGENDECOMPOSITION:
-Eigenvalues:
 [ 2.74371608e+01+0.j  1.94049006e+00+0.j -3.73673330e-16+0.j] -> [ 2.74371608e+01  1.94049006e+00 -3.73673330e-16]
-Eigenvectors:
 [[-0.12351467  0.56398356 -0.81649658]
 [-0.56722131 -0.71525752 -0.40824829]
 [-0.81425065  0.4127096   0.40824829]]

EIGENDECOMPOSITION USING SVD:
-Eigenvalues:
 [2.74439175e+01 1.94348595e+00 2.49533708e-30]
-Eigenvectors:
 [[-0.12361418  0.56396176 -0.81649658]
 [-0.56709509 -0.7153576  -0.40824829]
 [-0.81432346  0.41256593  0.40824829]] 

The two procedures approximately yield the same eigenvalues.
The two procedures approximately yield the same eigenvectors.


In [57]:
'''
    Principal Component Analysis (PCA) via dimension
    reduction. We begin with p = 3 features, hence the
    dimensionality
'''

# get lambda matrix
Lambda = np.diag(egl_real)

# select components
components = (0, 1, 2, (0, 1), (0, 2), (1, 2))

# compute variability for each component
variability = (abs(Lambda[0,0])/Lambda.trace(), abs(Lambda[1,1])/Lambda.trace(),
               abs(Lambda[2,2])/Lambda.trace(), abs(Lambda[0,0]+Lambda[1,1])/Lambda.trace(),
               abs(Lambda[0,0]+Lambda[2,2])/Lambda.trace(), abs(Lambda[0,0]+Lambda[2,2])/Lambda.trace())

for i, j in zip(components, variability):
    print("By selecting the component", i, "we retain %.5f of the total variability." % (j * 100))

By selecting the component 0 we retain 93.39467 of the total variability.
By selecting the component 1 we retain 6.60533 of the total variability.
By selecting the component 2 we retain 0.00000 of the total variability.
By selecting the component (0, 1) we retain 100.00000 of the total variability.
By selecting the component (0, 2) we retain 93.39467 of the total variability.
By selecting the component (1, 2) we retain 93.39467 of the total variability.


2\. PCA on a nD dataset

* Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normally distributed, with a standard deviation much smaller (e.g. a factor 20) than those used to generate the $x_1$ and $x_2$. Repeat the PCA procedure and compare the results with what you have obtained before.

3\. **Optional**: PCA on the MAGIC dataset

Perform a PCA on the magic04.data dataset.

In [ ]:
# get the dataset and its description on the proper data directory
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P data/
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P data/ 